In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re as re
import contractions
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, precision_score,mean_squared_error
from sklearn.metrics import precision_recall_fscore_support as score

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aakritinag/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/aakritinag/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
#! pip install bs4 # in case you don't have it installed
# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz

## Read Data

In [3]:
df = pd.read_table('amazon_reviews_us_Beauty_v1_00.tsv', sep='\t', low_memory=False, on_bad_lines='skip')
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,1797882,R3I2DHQBR577SS,B001ANOOOE,2102612,The Naked Bee Vitmin C Moisturizing Sunscreen ...,Beauty,5,0.0,0.0,N,Y,Five Stars,"Love this, excellent sun block!!",2015-08-31
1,US,18381298,R1QNE9NQFJC2Y4,B0016J22EQ,106393691,"Alba Botanica Sunless Tanning Lotion, 4 Ounce",Beauty,5,0.0,0.0,N,Y,Thank you Alba Bontanica!,The great thing about this cream is that it do...,2015-08-31
2,US,19242472,R3LIDG2Q4LJBAO,B00HU6UQAG,375449471,"Elysee Infusion Skin Therapy Elixir, 2oz.",Beauty,5,0.0,0.0,N,Y,Five Stars,"Great Product, I'm 65 years old and this is al...",2015-08-31
3,US,19551372,R3KSZHPAEVPEAL,B002HWS7RM,255651889,"Diane D722 Color, Perm And Conditioner Process...",Beauty,5,0.0,0.0,N,Y,GOOD DEAL!,I use them as shower caps & conditioning caps....,2015-08-31
4,US,14802407,RAI2OIG50KZ43,B00SM99KWU,116158747,Biore UV Aqua Rich Watery Essence SPF50+/PA+++...,Beauty,5,0.0,0.0,N,Y,this soaks in quick and provides a nice base f...,This is my go-to daily sunblock. It leaves no ...,2015-08-31


In [4]:
df.reset_index(drop=True, inplace=True)

## Keep Reviews and Ratings

In [5]:
enter = df [['star_rating','review_body']]
enter.head()

,star_rating,review_body
0,5,"Love this, excellent sun block!!"
1,5,The great thing about this cream is that it do...
2,5,"Great Product, I'm 65 years old and this is al..."
3,5,I use them as shower caps & conditioning caps....
4,5,This is my go-to daily sunblock. It leaves no ...


In [6]:
enter['star_rating'].unique()

array(['5', '4', '1', '3', '2', '2015-08-28', '2015-08-16', '2015-08-14',
       '2015-07-27', '2015-07-26', '2015-07-23', '2015-07-22', nan,
       '2015-06-14', '2015-06-02', '2015-04-14', '2015-04-09',
       '2015-04-08', '2015-04-03', '2015-04-02', '2015-04-01',
       '2015-03-31', '2015-03-30', '2015-03-18', '2015-02-28',
       '2015-02-10', '2014-12-30', '2014-12-03', '2014-10-09'],
      dtype=object)

In [7]:
enter['star_rating'] = enter['star_rating'].str.replace(r'\D', '')

/var/folders/m4/yx3n91cs37nf91mrrvjx9s2r0000gn/T/ipykernel_73283/955414285.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  enter['star_rating'] = enter['star_rating'].str.replace(r'\D', '')
/var/folders/m4/yx3n91cs37nf91mrrvjx9s2r0000gn/T/ipykernel_73283/955414285.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  enter['star_rating'] = enter['star_rating'].str.replace(r'\D', '')


In [8]:
# Drop rows that has NaN values 
enter = enter.dropna()

In [9]:
enter['star_rating'].unique()

array(['5', '4', '1', '3', '2'], dtype=object)

In [10]:
#keep only integers
enter['star_rating'] = enter['star_rating'].astype(int)

In [11]:
enter.head()

,star_rating,review_body
0,5,"Love this, excellent sun block!!"
1,5,The great thing about this cream is that it do...
2,5,"Great Product, I'm 65 years old and this is al..."
3,5,I use them as shower caps & conditioning caps....
4,5,This is my go-to daily sunblock. It leaves no ...


In [12]:
enter['star_rating'].unique()

array([5, 4, 1, 3, 2])

 ## We form three classes and select 20000 reviews randomly from each class.



In [13]:
def filter(x):
    if (x <= 2):
        return '1'
    if (x == 3):
        return '2'
    if (x > 3) :
        return '3'
enter['category'] = enter['star_rating'].apply(filter)

In [14]:
enter.head()

,star_rating,review_body,category
0,5,"Love this, excellent sun block!!",3
1,5,The great thing about this cream is that it do...,3
2,5,"Great Product, I'm 65 years old and this is al...",3
3,5,I use them as shower caps & conditioning caps....,3
4,5,This is my go-to daily sunblock. It leaves no ...,3


In [15]:
enter = enter.drop('star_rating', axis=1)

In [16]:
enter = enter.groupby('category').apply(lambda s: s.sample(20000))
enter

review_body category
category                                                                    
1        527980   The original shaver head that I got for this w...        1
         1010594                              would not order agian        1
         3343455  The scent just does not linger at all - feels ...        1
         3980377  This is supposed to be silk but it soaks and d...        1
         884048   This lotion came in a sealed bag, however, it ...        1
...                                                             ...      ...
3        3437399  This is a beautiful color! It looks like you h...        3
         1071421  Love Dermalogica. Purchased this for the prime...        3
         440382   Ok, this is something that works for me, and t...        3
         1738571  While it does take quite a while to receive it...        3
         2445570                                          Excellent        3

[60000 rows x 2 columns]

# Data Cleaning



In [17]:
#average length of reviews before cleaning
before_clean = enter['review_body'].str.len().mean()

In [18]:
# convert all reviews into lowercase
enter['review_body'] = enter['review_body'].str.lower()

In [19]:
enter

review_body category
category                                                                    
1        527980   the original shaver head that i got for this w...        1
         1010594                              would not order agian        1
         3343455  the scent just does not linger at all - feels ...        1
         3980377  this is supposed to be silk but it soaks and d...        1
         884048   this lotion came in a sealed bag, however, it ...        1
...                                                             ...      ...
3        3437399  this is a beautiful color! it looks like you h...        3
         1071421  love dermalogica. purchased this for the prime...        3
         440382   ok, this is something that works for me, and t...        3
         1738571  while it does take quite a while to receive it...        3
         2445570                                          excellent        3

[60000 rows x 2 columns]

In [20]:
#remove the HTML and URLs from the review
def remove(s):
    s=s.replace(r'<[^<>]*>', '')
    s=s.replace(r'http\S+', '').replace(r'www\S+', '')
    return s
enter['review_body']=enter['review_body'].apply(remove)
enter

review_body category
category                                                                    
1        527980   the original shaver head that i got for this w...        1
         1010594                              would not order agian        1
         3343455  the scent just does not linger at all - feels ...        1
         3980377  this is supposed to be silk but it soaks and d...        1
         884048   this lotion came in a sealed bag, however, it ...        1
...                                                             ...      ...
3        3437399  this is a beautiful color! it looks like you h...        3
         1071421  love dermalogica. purchased this for the prime...        3
         440382   ok, this is something that works for me, and t...        3
         1738571  while it does take quite a while to receive it...        3
         2445570                                          excellent        3

[60000 rows x 2 columns]

In [21]:
#Perform contractions on the reviews
enter['review_body'] = enter['review_body'].apply(lambda x: [contractions.fix(word) for word in x.split()])
enter['review_body'] = [' '.join(map(str, l)) for l in enter['review_body']]

In [22]:
#remove non alphabetical characyers and extra spaces
def remove(sentence):
    sentence=re.sub(r'[^a-zA-Z]',' ',sentence)
    return re.sub(' +', ' ',sentence)
enter['review_body'] = enter['review_body'].apply(remove)

In [23]:
enter

review_body category
category                                                                    
1        527980   the original shaver head that i got for this w...        1
         1010594                              would not order agian        1
         3343455  the scent just does not linger at all feels mo...        1
         3980377  this is supposed to be silk but it soaks and d...        1
         884048   this lotion came in a sealed bag however it is...        1
...                                                             ...      ...
3        3437399  this is a beautiful color it looks like you ha...        3
         1071421  love dermalogica purchased this for the primer...        3
         440382   ok this is something that works for me and tha...        3
         1738571  while it does take quite a while to receive it...        3
         2445570                                          excellent        3

[60000 rows x 2 columns]

In [24]:
# average length of reviews after cleaning 
after_clean=enter['review_body'].str.len().mean()

# Pre-processing

In [25]:
# average length of reviews before preprocessing 
before_preprocessing = enter['review_body'].str.len().mean()

## remove the stop words 

In [26]:
def remove_non_ascii(text): 
    return ''.join(i for i in text if ord(i)<128) 
 
enter['review_body'] = enter['review_body'].apply(remove_non_ascii) 

In [27]:
# stop = stopwords.words('english')
# enter['review_body'] = enter['review_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

## perform lemmatization  

In [28]:
lem = WordNetLemmatizer()

enter['review_body']=enter['review_body'].apply(lambda x: " ".join([lem.lemmatize(w) for w in nltk.word_tokenize(x)]))

enter

review_body category
category                                                                    
1        527980   the original shaver head that i got for this w...        1
         1010594                              would not order agian        1
         3343455  the scent just doe not linger at all feel more...        1
         3980377  this is supposed to be silk but it soak and dr...        1
         884048   this lotion came in a sealed bag however it is...        1
...                                                             ...      ...
3        3437399  this is a beautiful color it look like you hav...        3
         1071421  love dermalogica purchased this for the primer...        3
         440382   ok this is something that work for me and that...        3
         1738571  while it doe take quite a while to receive it ...        3
         2445570                                          excellent        3

[60000 rows x 2 columns]

In [29]:
# average length of reviews after preprocessing
after_preprocessing = enter['review_body'].str.len().mean()

In [30]:
print('Before and after preprocessing : '+str(before_preprocessing), ', '+str(after_preprocessing))

Before and after preprocessing : 260.90313333333336 , 256.39996666666667


# TF-IDF Feature Extraction

In [31]:
# from sklearn.feature_extraction.text import TfidfVectorizer
imp = TfidfVectorizer(ngram_range=(1,3))
x = imp.fit_transform(enter['review_body'])

In [32]:
# from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, enter['category'], test_size=0.2, random_state=42)

# Perceptron

In [33]:
from sklearn.linear_model import Perceptron
clf_per = Perceptron(random_state=42, tol=1e-3)
clf_per.fit(X_train,Y_train)

Perceptron(random_state=42)

In [34]:
pred_per= clf_per.predict(X_test)

In [35]:
print(confusion_matrix(Y_test,pred_per))

[[2863  829  271]
 [ 891 2400  723]
 [ 154  553 3316]]


In [36]:
target_names = ['class 1', 'class 2', 'class 3']
print(classification_report(Y_test, pred_per, target_names=target_names))

              precision    recall  f1-score   support

     class 1       0.73      0.72      0.73      3963
     class 2       0.63      0.60      0.62      4014
     class 3       0.77      0.82      0.80      4023

    accuracy                           0.71     12000
   macro avg       0.71      0.71      0.71     12000
weighted avg       0.71      0.71      0.71     12000



In [37]:
precision, recall, fscore, support = score(Y_test, pred_per)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))

precision: [0.7325998  0.63458488 0.76937355]
recall: [0.7224325  0.59790732 0.8242605 ]
fscore: [0.72748063 0.61570036 0.79587183]


In [38]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test,pred_per))

Accuracy: 0.7149166666666666


In [39]:
precision, recall, fscore, support = score(Y_test, pred_per)
sum_precision=0
sum_recall=0
sum_f1score=0
for i in range(3):
    print('class '+ str(i+1),'Precision:',precision[i],', Recall:', recall[i], ', F1 score:',fscore[i])
    sum_precision+= precision[i]
    sum_recall+= recall[i]
    sum_f1score+= fscore[i]
print('Averages : ' + str(sum_precision/3)+' , '+str(sum_recall/3)+' , '+str(sum_f1score/3))

class 1 Precision: 0.7325997952917093 , Recall: 0.7224325006308352 , F1 score: 0.7274806250794055
class 2 Precision: 0.6345848757271285 , Recall: 0.5979073243647235 , F1 score: 0.6157003591585428
class 3 Precision: 0.7693735498839908 , Recall: 0.8242605021128511 , F1 score: 0.795871834873395
Averages : 0.7121860736342761 , 0.7148667757028032 , 0.7130176063704478


# SVM

In [40]:
from sklearn.svm import LinearSVC
clf_svm= LinearSVC(random_state=0)
clf_svm.fit(X_train,Y_train)

LinearSVC(random_state=0)

In [41]:
pred_svm=clf_svm.predict(X_test)

In [42]:
target_names = ['class 1', 'class 2', 'class 3']
print(classification_report(Y_test, pred_svm, target_names=target_names))

              precision    recall  f1-score   support

     class 1       0.74      0.76      0.75      3963
     class 2       0.66      0.62      0.64      4014
     class 3       0.81      0.83      0.82      4023

    accuracy                           0.74     12000
   macro avg       0.74      0.74      0.74     12000
weighted avg       0.74      0.74      0.74     12000



In [43]:
precision, recall, fscore, support = score(Y_test, pred_svm)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))

precision: [0.73898139 0.66121803 0.81251516]
recall: [0.76154428 0.62481315 0.83271191]
fscore: [0.7500932  0.64250032 0.82248957]


In [44]:
print("Accuracy:",metrics.accuracy_score(Y_test,pred_svm))

Accuracy: 0.7396666666666667


In [45]:
precision, recall, fscore, support = score(Y_test, pred_svm)
sum_precision=0
sum_recall=0
sum_f1score=0
for i in range(3):
    print('class '+ str(i+1),'Precision:',precision[i],', Recall:', recall[i], ', F1 score:',fscore[i])
    sum_precision+= precision[i]
    sum_recall+= recall[i]
    sum_f1score+= fscore[i]
print('Averages : ' + str(sum_precision/3)+' , '+str(sum_recall/3)+' , '+str(sum_f1score/3) )

class 1 Precision: 0.7389813907933399 , Recall: 0.761544284632854 , F1 score: 0.7500932024356904
class 2 Precision: 0.6612180332190878 , Recall: 0.624813153961136 , F1 score: 0.6425003202254388
class 3 Precision: 0.8125151588649042 , Recall: 0.8327119065374099 , F1 score: 0.8224895654308865
Averages : 0.7375715276257773 , 0.7396897817104665 , 0.7383610293640053


# Logistic Regression

In [46]:
from sklearn.linear_model import LogisticRegression

clf_log= LogisticRegression(random_state=42, max_iter=10000)
clf_log.fit(X_train,Y_train)

LogisticRegression(max_iter=10000, random_state=42)

In [47]:
pred_log=clf_log.predict(X_test)

In [48]:
target_names = ['class 1', 'class 2', 'class 3']
print(classification_report(Y_test, pred_log, target_names=target_names))

              precision    recall  f1-score   support

     class 1       0.74      0.75      0.75      3963
     class 2       0.65      0.64      0.65      4014
     class 3       0.81      0.81      0.81      4023

    accuracy                           0.74     12000
   macro avg       0.74      0.74      0.74     12000
weighted avg       0.74      0.74      0.74     12000



In [49]:
precision, recall, fscore, support = score(Y_test, pred_log)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))

precision: [0.73688101 0.65460107 0.81358885]
recall: [0.75473126 0.63976084 0.81257768]
fscore: [0.74569933 0.64709588 0.81308295]


In [50]:
print("Accuracy:",metrics.accuracy_score(Y_test,pred_log))

Accuracy: 0.7356666666666667


In [51]:
precision, recall, fscore, support = score(Y_test, pred_log)
sum_precision=0
sum_recall=0
sum_f1score=0
for i in range(3):
    print('class '+ str(i+1),'Precision:',precision[i],', Recall:', recall[i], ', F1 score:',fscore[i])
    sum_precision+= precision[i]
    sum_recall+= recall[i]
    sum_f1score+= fscore[i]
print('Averages : ' + str(sum_precision/3)+' , '+str(sum_recall/3)+' , '+str(sum_f1score/3) )

class 1 Precision: 0.7368810051736882 , Recall: 0.7547312641937926 , F1 score: 0.7456993268511594
class 2 Precision: 0.6546010706092277 , Recall: 0.6397608370702541 , F1 score: 0.6470958800554366
class 3 Precision: 0.813588850174216 , Recall: 0.8125776783494905 , F1 score: 0.8130829498818554
Averages : 0.7350236419857107 , 0.7356899265378457 , 0.7352927189294838


# Naive Bayes

In [52]:
from sklearn.naive_bayes import MultinomialNB

clf_naive = MultinomialNB()
clf_naive.fit(X_train, Y_train)

MultinomialNB()

In [53]:
pred_naive = clf_naive.predict(X_test)

In [54]:
target_names = ['class 1', 'class 2', 'class 3']
print(classification_report(Y_test, pred_naive, target_names=target_names))

              precision    recall  f1-score   support

     class 1       0.73      0.76      0.74      3963
     class 2       0.62      0.70      0.65      4014
     class 3       0.88      0.73      0.80      4023

    accuracy                           0.73     12000
   macro avg       0.74      0.73      0.73     12000
weighted avg       0.74      0.73      0.73     12000



In [55]:
print("Accuracy:",metrics.accuracy_score(Y_test,pred_naive))

Accuracy: 0.7271666666666666


In [56]:
precision, recall, fscore, support = score(Y_test, pred_naive)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))

precision: [0.72592772 0.61658946 0.87858852]
recall: [0.75523593 0.6963129  0.73030077]
fscore: [0.74029186 0.65403065 0.79761097]


In [57]:
precision, recall, fscore, support = score(Y_test, pred_naive)
sum_precision=0
sum_recall=0
sum_f1score=0
for i in range(3):
    print('class '+ str(i+1),'Precision:',precision[i],', Recall:', recall[i], ', F1 score:',fscore[i])
    sum_precision+= precision[i]
    sum_recall+= recall[i]
    sum_f1score+= fscore[i]
print('Averages : ' + str(sum_precision/3)+' , '+str(sum_recall/3)+' , '+str(sum_f1score/3))

class 1 Precision: 0.7259277225321368 , Recall: 0.7552359323744637 , F1 score: 0.7402918624783577
class 2 Precision: 0.6165894551069931 , Recall: 0.6963129048330842 , F1 score: 0.654030654030654
class 3 Precision: 0.8785885167464115 , Recall: 0.7303007705692269 , F1 score: 0.79761096782951
Averages : 0.7403685647951805 , 0.7272832025922583 , 0.7306444947795073
